# environment sanity & base config

In [ ]:
%pip intall requirements.txt

# %pip install sounddevice
# %pip install python-dotenv
# %pip install scipy
# %npm install @elevenlabs/react
# %pip install websockets

In [4]:
import os
import sys
import platform
from pathlib import Path
from datetime import datetime

In [5]:
try:
    import sounddevice as sd  #microphone capture
    sd_available = True
except Exception as e:
    sd = None
    sd_available = False

try:
    from dotenv import load_dotenv  # load .env file
    dotenv_available = True
except Exception:
    dotenv_available = False
    

print("sounddevice available:", sd_available)
print("dotenv available:", dotenv_available)

sounddevice available: True
dotenv available: True


Load and verify API KEY

In [6]:
def mask_secret(s: str, keep: int = 6) -> str:
    """mask secret string, keep first and last `keep` characters visible"""
    if not s:
        return "<EMPTY>"
    if len(s) <= keep:
        return "*" * len(s)
    return s[:keep] + "*" * (len(s) - 2*keep) + s[-keep:]

In [7]:
if dotenv_available:
    load_dotenv()  
else:
    print("[WARN] python-dotenv not found, .env loader not found）")

ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY", "")

# assertion
assert ELEVENLABS_API_KEY, "ELEVENLABS_API_KEY not found, please set it in .env。"

experimental defaults

In [8]:
MODEL_ID = "scribe_v2_realtime"          # 模型 ID（model_id）
LANGUAGE_CODE = "en"                      # 语言（language_code），英文课堂/会议/面试
AUDIO_FORMAT = "pcm_16000"                # 音频格式（audio_format）：16kHz/16-bit/mono/little-endian
CHUNK_MS = 200                            # 分块时长（chunk duration ms）：初始建议 200ms
COMMIT_STRATEGY = "vad"                   # 分段策略（commit strategy）：'vad' 或 'manual'

# VAD 参数（仅在 commit_strategy = 'vad' 时有效）
VAD_SILENCE_THRESHOLD_SECS = 1.5          # 达到静音阈值后自动提交（silence threshold）
VAD_THRESHOLD = 0.4                       # 灵敏度（0.1-0.9，越低越敏感）
MIN_SPEECH_DURATION_MS = 100              # 最短语音持续（ms）
MIN_SILENCE_DURATION_MS = 100             # 最短静音持续（ms）

# WebSocket 端点（WebSocket endpoint）
WS_URL = f"wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id={MODEL_ID}"

test output dir(experimental)

In [9]:
RUNS_ROOT = Path.cwd().parent / "runs" if Path.cwd().name == "notebooks" else Path.cwd() / "runs"
RUN_DIR = RUNS_ROOT / datetime.now().strftime("%Y-%m-%d_%H%M%S")
RUN_DIR.mkdir(parents=True, exist_ok=True)

Print environment summary

In [10]:
print("== Environment Summary ==")
print(f"Project Root        : {Path.cwd().resolve().parent if Path.cwd().name == 'notebooks' else Path.cwd().resolve()}")
print(f"Python              : {sys.version.split()[0]} on {platform.system()} {platform.release()} ({platform.machine()})")
print(f".env ELEVENLABS_KEY : {mask_secret(ELEVENLABS_API_KEY, keep=6)}")
print(f"Model ID            : {MODEL_ID}")
print(f"Language Code       : {LANGUAGE_CODE}")
print(f"Audio Format        : {AUDIO_FORMAT}")
print(f"Chunk (ms)          : {CHUNK_MS}")
print(f"Commit Strategy     : {COMMIT_STRATEGY}")
if COMMIT_STRATEGY.lower() == "vad":
    print(f"  VAD silence secs  : {VAD_SILENCE_THRESHOLD_SECS}")
    print(f"  VAD threshold     : {VAD_THRESHOLD}")
    print(f"  min speech (ms)   : {MIN_SPEECH_DURATION_MS}")
    print(f"  min silence (ms)  : {MIN_SILENCE_DURATION_MS}")

print(f"WS URL              : {WS_URL}")
print(f"Run Output Dir      : {RUN_DIR.resolve()}")

if sd_available:
    try:
        ver = sd.get_portaudio_version()
        default_in = sd.default.device[0] if sd.default.device is not None else None
        print("Sounddevice         : available")
        print(f"  PortAudio version : {ver}")
        print(f"  Default input dev : {default_in}")
    except Exception as e:
        print("Sounddevice         : available but cannot query details ->", repr(e))
else:
    print("Sounddevice         : NOT available (install 'sounddevice' and ensure PortAudio is present)")

print("✅ Cell 1 successfully Initialized")

== Environment Summary ==
Project Root        : /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG
Python              : 3.10.19 on Darwin 24.6.0 (arm64)
.env ELEVENLABS_KEY : <sk_c0*****************************************3c80c>
Model ID            : scribe_v2_realtime
Language Code       : en
Audio Format        : pcm_16000
Chunk (ms)          : 200
Commit Strategy     : vad
  VAD silence secs  : 1.5
  VAD threshold     : 0.4
  min speech (ms)   : 100
  min silence (ms)  : 100
WS URL              : wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime
Run Output Dir      : /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/runs/2025-11-12_205603
Sounddevice         : available
  PortAudio version : (1246976, 'PortAudio V19.7.0-devel, revision unknown')
  Default input dev : 2
✅ Cell 1 successfully Initialized


Microphone sanity check + PCM format validation

In [11]:
import numpy as np
import sounddevice as sd
from scipy.io.wavfile import write
from pathlib import Path

Set paprameters

In [12]:
SAMPLE_RATE = 16000           # 采样率 (Hz)
CHANNELS = 1                  # 单声道 (mono)
DURATION = 5                  # 录制时长 (秒)
TEST_FILENAME = RUN_DIR / "runs/manual_test.wav"

Reccord

In [ ]:
# Cell — 使用现成的测试 WAV文件（manual_text_16k.wav）做分块 + base64

import base64
import numpy as np
from scipy.io import wavfile
from pathlib import Path

project_root = Path("/Users/dwg/CODE/AI_Projects/realtime_transcriber_WG")
wav_path = project_root / "runs" / "manual_test_16k.wav"

assert wav_path.exists(), f"找不到 WAV 文件: {wav_path}"

print(f"🎧 使用测试音频: {wav_path.resolve()}")

# 读取 WAV
sr, data = wavfile.read(wav_path)
print(f"  采样率 (sample_rate): {sr} Hz")
print(f"  数据形状 (shape)    : {data.shape}, dtype={data.dtype}")

# 保证 mono（单声道）
if data.ndim == 2 and data.shape[1] > 1:
    print("  检测到多声道，转换为 mono（简单平均）")
    data_mono = data.mean(axis=1).astype(np.int16)
else:
    data_mono = data.astype(np.int16).flatten()

duration_sec = len(data_mono) / sr
print(f"  时长 (duration)     : {duration_sec:.2f} 秒")

max_val, min_val = data_mono.max(), data_mono.min()
mean_abs = np.mean(np.abs(data_mono))
print(f"  振幅范围 (max/min)  : {max_val} / {min_val}")
print(f"  平均绝对值 (mean|x|): {mean_abs:.2f}")

if mean_abs == 0:
    print("⚠️ 警告：该音频看起来是静音（mean_abs=0），转写结果可能为空，"
          "但仍可用于测试 WebSocket 流程。")



🎧 使用测试音频: /Users/dwg/CODE/AI_Projects/realtime_transcriber_WG/runs/manual_test_16k.wav
  采样率 (sample_rate): 16000 Hz
  数据形状 (shape)    : (166528,), dtype=int16
  时长 (duration)     : 10.41 秒
  振幅范围 (max/min)  : 12402 / -10723
  平均绝对值 (mean|x|): 756.69


In [37]:
# 分块（chunking），使用前面 Cell1 中定义的 CHUNK_MS
chunk_samples = int(sr * CHUNK_MS / 1000)
print(f"📦 每块样本数 (samples per chunk for {CHUNK_MS}ms): {chunk_samples}")

def generate_pcm_chunks(int16_array: np.ndarray, samples_per_chunk: int):
    n = len(int16_array)
    for start in range(0, n, samples_per_chunk):
        end = min(start + samples_per_chunk, n)
        chunk = int16_array[start:end]
        yield chunk.astype('<i2').tobytes()  # 小端 int16 PCM

chunks_bytes = list(generate_pcm_chunks(data_mono, chunk_samples))
print(f"✅ 总块数 (total chunks): {len(chunks_bytes)}")

# 预览首块的 base64
if chunks_bytes:
    first_chunk = chunks_bytes[0]
    b64_preview = base64.b64encode(first_chunk).decode("ascii")
    print(f"  首块字节长度 (bytes len)      : {len(first_chunk)}")
    print(f"  首块 base64 长度 (base64 len): {len(b64_preview)}")
    print(f"  首块 base64 预览 (前80字符)   : {b64_preview[:80]}...")
else:
    print("⚠️ 警告：没有生成任何 chunk。")

print("✅ 分块 + base64 预处理完成，可以用于后续 WebSocket 推流实验。")

📦 每块样本数 (samples per chunk for 200ms): 3200
✅ 总块数 (total chunks): 53
  首块字节长度 (bytes len)      : 6400
  首块 base64 长度 (base64 len): 8536
  首块 base64 预览 (前80字符)   : CQAAAPn////+//f/AAD+/wEABQD7//3/BAADAAIAAwAAAAQABgD8//n/AAD///7/BAABAAAA/P///wAA...
✅ 分块 + base64 预处理完成，可以用于后续 WebSocket 推流实验。


In [38]:
import asyncio
import base64
import json
from typing import List
try:
    import websockets
except ImportError as e:
    raise ImportError(
        "未找到 websockets 库，请先安装：\n"
        "  pip install websockets\n"
        "然后重新运行本 Cell。"
    ) from e

# 确保我们有 chunks_bytes 和 sr（采样率）
try:
    len(chunks_bytes)
except NameError:
    raise RuntimeError("chunks_bytes 未定义，请先运行分块 + base64 的那个 Cell。")

In [40]:
STREAM_SAMPLE_RATE = 16000  # 如果你以后换了采样率，这里要和音频一致

print(f"将使用 {len(chunks_bytes)} 个音频块推流，sample_rate={STREAM_SAMPLE_RATE}, chunk_ms={CHUNK_MS}ms")
print("连接 WebSocket 并开始转写...")

async def send_audio(ws, chunks: List[bytes], sample_rate: int):
    """
    按固定节奏发送音频块到 WebSocket。
    模拟真实时间：每个块之间 sleep CHUNK_MS/1000 秒。
    最后发送一次 commit=True，触发 committed_transcript。
    """
    chunk_interval = CHUNK_MS / 1000.0  # 秒
    print(f"🛰 发送任务启动，每 {chunk_interval:.3f}s 发送 1 个 chunk...")

    for idx, chunk in enumerate(chunks):
        audio_b64 = base64.b64encode(chunk).decode("ascii")
        payload = {
            "message_type": "input_audio_chunk",
            "audio_base_64": audio_b64,
            "commit": False,
            "sample_rate": sample_rate,
        }
        await ws.send(json.dumps(payload))
        # 打个点方便看进度
        if (idx + 1) % 10 == 0 or idx == len(chunks) - 1:
            print(f"   -> 已发送 {idx + 1}/{len(chunks)} 块")

        await asyncio.sleep(chunk_interval)

    # 所有块发完后，稍微等一下，再发 commit
    await asyncio.sleep(0.5)
    print("📝 所有块已发送，发送最终 commit 请求...")

    final_payload = {
        "message_type": "input_audio_chunk",
        "audio_base_64": "",   # 无需再次发送音频
        "commit": True,
        "sample_rate": sample_rate,
    }
    await ws.send(json.dumps(final_payload))
    print("✅ commit 消息已发送。")


async def receive_transcripts(ws):
    """
    从 WebSocket 接收消息并根据 message_type 打印。
    我们主要关心：
      - session_started
      - partial_transcript
      - committed_transcript
      - committed_transcript_with_timestamps
    """
    print("📡 接收任务启动，等待转录结果...")
    try:
        while True:
            try:
                msg = await asyncio.wait_for(ws.recv(), timeout=20.0)
            except asyncio.TimeoutError:
                print("⏱ 20 秒未收到新消息，结束接收任务。")
                break

            data = json.loads(msg)
            msg_type = data.get("message_type")

            if msg_type == "session_started":
                print("🚀 会话已建立 (session_started):")
                print("   ", json.dumps(data, ensure_ascii=False)[:200], "...")
            elif msg_type == "partial_transcript":
                text = data.get("text", "")
                if text:
                    print(f"🟡 Partial: {text}")
            elif msg_type == "committed_transcript":
                text = data.get("text", "")
                print(f"🟢 Committed: {text}")
            elif msg_type == "committed_transcript_with_timestamps":
                text = data.get("text", "")
                words = data.get("words", [])
                print(f"📌 Committed + timestamps: {text}")
                if words:
                    print(f"   (示例前 3 个词): {words[:3]}")
                # 收到带时间戳的 committed，说明这一段已经完整，结束接收
                print("✅ 已收到 committed_transcript_with_timestamps，结束接收任务。")
                break
            elif msg_type in ("auth_error", "quota_exceeded", "transcriber_error", "input_error", "error"):
                print(f"❌ 错误消息 ({msg_type}): {data}")
                # 出错时直接结束
                break
            else:
                # 其他类型消息直接打印简要信息
                print(f"ℹ️ 收到其他消息类型: {msg_type}")
                print("   ", json.dumps(data, ensure_ascii=False)[:200], "...")
    except Exception as e:
        print("❌ 接收过程中出现异常:", repr(e))


async def main_realtime_test():
    """
    建立 WebSocket 连接，启动发送和接收两个协程。
    """
    headers = {
        "xi-api-key": ELEVENLABS_API_KEY,
    }

    print(f"🔌 连接到 WebSocket: {WS_URL}")
    async with websockets.connect(WS_URL, additional_headers=headers) as ws:
        # 先等待 session_started 消息（有些情况下服务器会主动先发）
        try:
            initial_msg = await asyncio.wait_for(ws.recv(), timeout=5.0)
            data = json.loads(initial_msg)
            print("📥 初始消息:", data.get("message_type"), "|",
                  json.dumps(data, ensure_ascii=False)[:200], "...")
        except asyncio.TimeoutError:
            print("⚠️ 5 秒内未收到初始消息（session_started），继续发送音频。")

        # 并行执行：发送音频 + 接收转录
        sender = asyncio.create_task(send_audio(ws, chunks_bytes, STREAM_SAMPLE_RATE))
        receiver = asyncio.create_task(receive_transcripts(ws))

        # 等两个任务都结束
        await asyncio.gather(sender, receiver)

    print("🔚 WebSocket 连接已关闭。实验结束。")


# 在 notebook 中运行 asyncio
try:
    await main_realtime_test()  # 如果你的 Jupyter 支持直接 await（IPython 8+）
except SyntaxError:
    # 某些环境不允许在 cell 顶层用 await，就退回 asyncio.run
    import asyncio
    asyncio.run(main_realtime_test())

将使用 53 个音频块推流，sample_rate=16000, chunk_ms=200ms
连接 WebSocket 并开始转写...
🔌 连接到 WebSocket: wss://api.elevenlabs.io/v1/speech-to-text/realtime?model_id=scribe_v2_realtime
📥 初始消息: auth_error | {"message_type": "auth_error", "error": "You must be authenticated to use this endpoint."} ...
🛰 发送任务启动，每 0.200s 发送 1 个 chunk...
📡 接收任务启动，等待转录结果...
❌ 接收过程中出现异常: ConnectionClosedOK(Close(code=1000, reason=''), Close(code=1000, reason=''), True)


ConnectionClosedOK: received 1000 (OK); then sent 1000 (OK)